In [ ]:
import os
from functools import partial
from time import strftime

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

plt.style.use("dark_background")
tf.config.list_physical_devices("GPU")

In [ ]:
# Get project files
#!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip
#!unzip cats_and_dogs.zip

PATH = "cats_and_dogs"

train_dir = os.path.join(PATH, "train")
validation_dir = os.path.join(PATH, "validation")
test_dir = os.path.join(PATH, "test")

# Get number of files in each directory. The train and validation directories
# each have the subdirectories "dogs" and "cats".
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])
total_test = len(os.listdir(test_dir))

# Variables for pre-processing and training.
batch_size = 32
epochs = 15
IMG_HEIGHT = 128
IMG_WIDTH = IMG_HEIGHT

In [ ]:
# 3
CLASS_LABELS = ("cat", "dog")


def parse_image(image_path):

    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)

    filename = tf.strings.split(image_path, "/")[-1]
    class_name = tf.strings.split(filename, ".")[0]
    label_index = tf.cast(tf.equal(class_name, CLASS_LABELS[1]), tf.int32)
    return image, label_index


preprocessing_layer = tf.keras.Sequential(
    [
        tf.keras.layers.Rescaling(scale=1.0 / 255),
        tf.keras.layers.Resizing(IMG_HEIGHT, IMG_WIDTH, pad_to_aspect_ratio=True),
    ]
)


def get_dataset(directory, cache=False, shuffle=False, batch=True):
    ds = (
        tf.data.Dataset.list_files([directory + _ for _ in ["/*/*.jpg", "/*.jpg"]])
        .map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
        .map(
            lambda image, label_index: (preprocessing_layer(image), label_index),
            num_parallel_calls=tf.data.AUTOTUNE,
        )
    )

    if cache:
        ds = ds.cache()
    if shuffle:
        ds = ds.shuffle(2_000)
    if batch:
        ds = ds.batch(batch_size)
    return ds.prefetch(tf.data.AUTOTUNE)


train_ds = get_dataset(train_dir, cache=True, shuffle=True)
validation_ds = get_dataset(validation_dir, cache=True)
test_ds = get_dataset(test_dir)

In [ ]:
# 4
def plot_images(images_arr, probabilities=False):
    fig, axes = plt.subplots(
        ncols=len(images_arr), nrows=1, figsize=(len(images_arr) * 3, 5)
    )
    if probabilities is False:
        for img, ax in zip(images_arr, axes):
            ax.imshow(img)
            ax.axis("off")
    else:
        for img, probability, ax in zip(images_arr, probabilities, axes):
            ax.imshow(img, vmin=0, vmax=1)
            ax.axis("off")
            if probability > 0.5:
                ax.set_title("%.2f" % (probability * 100) + "% dog")
            else:
                ax.set_title("%.2f" % ((1 - probability) * 100) + "% cat")
    plt.show()

In [ ]:
# 5
data_augmentation_layer = tf.keras.Sequential(
    [
        tf.keras.layers.RandomBrightness(0.1, value_range=(0, 1)),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
        tf.keras.layers.RandomRotation(0.05),
    ]
)

In [ ]:
# 6
sample_training_images, sample_training_labels = next(iter(train_ds))
sample_training_images, sample_training_labels = (
    sample_training_images[:5],
    sample_training_labels[:5].numpy(),
)
plot_images(sample_training_images, sample_training_labels)

augmented_images = data_augmentation_layer(sample_training_images, training=True)

plot_images(augmented_images, sample_training_labels)

In [ ]:
# 7

DefaultConv2D = partial(
    tf.keras.layers.Conv2D,
    kernel_size=3,
    padding="same",
    activation="relu",
    kernel_initializer="he_normal",
)

DefaultDense = partial(
    tf.keras.layers.Dense,
    activation="relu",
    kernel_initializer="he_normal",
)


def make_model(learning_rate=0.01):
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
            # data_augmentation_layer,
            DefaultConv2D(filters=64, kernel_size=7),
            tf.keras.layers.MaxPool2D(),
            DefaultConv2D(filters=128),
            DefaultConv2D(filters=128),
            tf.keras.layers.MaxPool2D(),
            DefaultConv2D(filters=256),
            DefaultConv2D(filters=256),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.GlobalAveragePooling2D(),
            DefaultDense(units=128),
            # tf.keras.layers.Dropout(0.5),
            DefaultDense(units=64),
            # tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(units=1, activation="sigmoid"),
        ]
    )

    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.SGD(
            learning_rate=learning_rate, momentum=0.9, nesterov=True
        ),
        metrics=["accuracy"],
    )

    return model


model = make_model()
model.summary()

In [ ]:
class ExponentialLearningRate(tf.keras.callbacks.Callback):
    def __init__(self, starting_lr, end_lr, n_iter, log_dir=None):
        self.factor = (end_lr / starting_lr) ** (1 / n_iter)
        self.starting_lr = starting_lr
        self.end_lr = end_lr
        self.learning_rates, self.losses = [], []
        self.file_writer = (
            tf.summary.create_file_writer(log_dir + "/exp_learning_rate")
            if log_dir
            else None
        )

    def on_train_begin(self, logs=None):
        self.model.optimizer.learning_rate = self.starting_lr

    def on_train_batch_end(self, batch, logs=None):
        lr = self.model.optimizer.learning_rate.numpy()

        if logs:
            self.learning_rates.append(lr)
            self.losses.append(logs["loss"])

        if logs and self.file_writer:
            with self.file_writer.as_default():
                tf.summary.scalar(
                    "Learning Rate",
                    lr,
                    step=batch,
                )
                tf.summary.scalar(
                    "Loss",
                    logs["loss"],
                    step=batch,
                )

        new_lr = lr * self.factor
        if new_lr > self.end_lr:
            self.model.stop_training = True
            print("\nReached end learning rate, stopping training")
        else:
            self.model.optimizer.learning_rate = new_lr

In [ ]:
exponential_learning_rate_cb = ExponentialLearningRate(1e-4, 1, 100)

history = model.fit(
    train_ds,
    epochs=100,
    callbacks=[exponential_learning_rate_cb],
)

In [ ]:
fig, ax = plt.subplots()

ax.plot(
    exponential_learning_rate_cb.learning_rates,
    exponential_learning_rate_cb.losses,
)
ax.set_xscale("log")
ax.set_xlabel("Learning Rate")
ax.set_ylabel("Loss")
ax.set_ylim(min(exponential_learning_rate_cb.losses) * 0.99, 1)
ax.grid(True)

plt.show()

In [ ]:
# 8
model = make_model(learning_rate=0.2)

history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds,
    callbacks=[
        tf.keras.callbacks.TensorBoard(
            f"logs/cat_dog/{strftime('run_%H_%M_%S')}",
        ),
    ],
)

In [ ]:
# 9
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
# 10
probabilities = model.predict(test_ds)

In [ ]:
# 11
answers = [
    1,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    1,
    1,
    0,
    1,
    0,
    1,
    0,
    1,
    1,
    0,
    1,
    1,
    0,
    0,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    0,
    0,
    0,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    0,
    1,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
]

correct = 0

for probability, answer in zip(probabilities, answers):
    if np.round(probability) == answer:
        correct += 1

percentage_identified = (correct / len(answers)) * 100

passed_challenge = percentage_identified >= 63

print(
    f"Your model correctly identified {round(percentage_identified, 2)}% of the images of cats and dogs."
)

if passed_challenge:
    print("You passed the challenge!")
else:
    print(
        "You haven't passed yet. Your model should identify at least 63% of the images. Keep trying. You will get it!"
    )